<a href="https://colab.research.google.com/github/bhattacharya5/DLOps/blob/main/M21AIE211_DLOps_ClassAssignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Class Assignment 1: CNN on CIFAR-10 data.

Total Marks: 30


Import the necessary python packages and libraries in the following cell.

In [15]:
## Import the libraries here in this cell
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

from PIL import Image
from tempfile import TemporaryDirectory

plt.ion()   # interactive mode

from torchvision.models import resnet18
import seaborn as sns

%matplotlib inline

Q1 (a) Print the torch and torchvision version [Marks: 1+1]

In [2]:
# Your code here
print("PyTorch version:", torch.__version__)
print("TorchVision version:", torchvision.__version__)

PyTorch version: 2.1.0+cu121
TorchVision version: 0.16.0+cu121


Q1 (b) Set the device to cuda, if available. [Marks: 1]

In [3]:
# device = // Your code here
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Q1 (c) Transform or preprocess the data utilizing the following operations. [Marks: 1x5 = 5]

*   Normalize data
*   Resize each image to 224 x 224
*   Add Color Jitter with hue and saturation as 0.5
*   Introduce Random horizontal flips
*   Rotate images about 20 degrees


In [4]:
# transform = // Your code here

# Data transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(hue=0.5, saturation=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

Q2 (a) Train and test Resnet18 on Even numbered Classes in CIFAR10 Dataset. [Marks: 5]

In [5]:
# Filter only even-numbered classes
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Your code here
#HINT: Use torch utils Subset class to create a subset of the dataset
even_classes = [i for i in range(0, 10, 2)]

trainset = torch.utils.data.Subset(trainset, [i for i in range(len(trainset)) if trainset[i][1] in even_classes])
testset = torch.utils.data.Subset(testset, [i for i in range(len(testset)) if testset[i][1] in even_classes])


100%|██████████| 170498071/170498071 [00:10<00:00, 15812745.49it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [9]:
model = resnet18(weights='DEFAULT').to(device)
model.fc = nn.Linear(model.fc.in_features, len(even_classes)).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 160MB/s]


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Q2 (b) Complete the code for backpropagation inside the train function. [Marks: 5]


In [12]:
def train(model, trainloader, criterion, optimizer, device):

    model.train()

    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in trainloader:

        inputs, labels = inputs.to(device), labels.to(device)

        # Complete the code
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        train_loss += loss.item()

        _, pred = outputs.max(1)
        total_train += labels.size(0)
        correct_train += pred.eq(labels).sum().item()

    train_acc = 100 * correct_train / total_train

    return train_loss/len(trainloader), train_acc

Q2 (c) Complete the test function to test out the model. [Marks: 5]

In [13]:
def test(model, testloader, criterion, device):
    model.eval()
    correct_test = 0
    total_test = 0

    # Your code here
    with torch.no_grad():
      for inputs, labels in testloader:
          inputs, labels = inputs.to(device), labels.to(device)

          # Forward pass
          outputs = model(inputs)
          _, pred = outputs.max(1)

          total_test += labels.size(0)
          correct_test += pred.eq(labels).sum().item()

    test_accuracy = 100 * correct_test / total_test

    return test_accuracy

Q3 (a) Complete the code to plot the train and test accuracies [Marks: 2]

In [14]:
def plot_accuracies(train_accuracies, test_accuracies, epochs):

    # Your code here
    plt.plot(range(1, epochs + 1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, epochs + 1), test_accuracies, label='Test Accuracy')

    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Train and Test Accuracies vs Epochs')
    plt.legend()
    plt.show()

Q3 (b) Complete the code to plot a Confusion Matrix (without sklearn) [Marks: 3]

In [16]:
def plot_confusion_matrix(model, dataloader, class_names, device):
    model.eval()
    all_labels = []
    all_predictions = []

    num_classes = len(class_names)
    confusion_matrix = torch.zeros(num_classes, num_classes)

    # Your code here
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, predictions = outputs.max(1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    # Calculate confusion matrix
    for i in range(len(all_labels)):
        confusion_matrix[all_labels[i], all_predictions[i]] += 1

    plt.figure(figsize=(num_classes, num_classes))
    sns.heatmap(confusion_matrix, annot=True, fmt=".0f", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()


Q3 (c) Complete the following code block to train and test the model. [Marks: 2]

In [17]:
epochs = 10
train_accuracies = []
test_accuracies = []

for epoch in range(epochs):

  # Complete the loop
  for epoch in range(epochs):
      # Train the model
      train_loss, train_acc = train(model, trainloader, criterion, optimizer, device)
      train_accuracies.append(train_acc)

      # Test the model
      test_acc = test(model, testloader, criterion, device)
      test_accuracies.append(test_acc)

      print(f'Epoch {epoch + 1} | Train Loss: {train_loss:.3f} | Train Accuracy: {train_acc:.3f}% | Test Accuracy: {test_acc:.3f}%')

plot_accuracies(train_accuracies, test_accuracies, epochs)
class_names = ['0', '2', '4', '6', '8']
plot_confusion_matrix(model, testloader, class_names, device)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
